# Single Cell Mapper

## Libraries

Run the chunk below to import the required libraries

In [146]:
import numpy as np
import pandas as pd
import scanpy as sc
import h5py
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans
from sklearn.preprocessing import *
import random
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import scipy.sparse
import scipy.io
from scipy import * 
from sklearn.decomposition import PCA

## Single Cell Mapper Autoencoder

Run the chunk below to create the autoencoder class that will be trained later

In [192]:
class RNA_seq_ae:
    
    def __init__(self, ref_file, query_file):
        self.ref_ann, self.query_ann = data_import_sc(ref_file, query_file)
        self.ref_data = self.ref_ann.X
        self.query_data = self.query_ann.X
        self.ref_data_t = np.transpose(self.ref_data)
        self.query_data_t = np.transpose(self.query_data)

    def processed_data(self):
        return self.ref_ann, self.query_ann

    def processed_data_t(self):
        return self.ref_ann_t, self.query_ann_t
        
    class Autoencoder(nn.Module):
        
        def __init__(self, ref_len, query_len, r_hd, q_hd, r_bd, q_bd, c_hd, c_bd):
    
            super(RNA_seq_ae.Autoencoder, self).__init__()

            self.ref_encoder = nn.Sequential(
                nn.Linear(ref_len, r_hd),
                nn.ReLU(),
                nn.Linear(r_hd, r_bd))

            self.query_encoder = nn.Sequential(
                nn.Linear(query_len, q_hd),
                nn.ReLU(),
                nn.Linear(q_hd, q_bd))

            self.combined_encoder = nn.Sequential(
                nn.Linear(q_bd + r_bd, c_hd),
                nn.ReLU(),
                nn.Linear(c_hd, c_bd))
            
            self.ref_decoder = nn.Sequential(
                nn.Linear(c_bd, r_hd),
                nn.ReLU(),
                nn.Linear(r_hd, ref_len),
                nn.Tanhshrink())

            self.query_decoder = nn.Sequential(
                nn.Linear(c_bd, q_hd),
                nn.ReLU(),
                nn.Linear(q_hd, query_len),
                nn.Tanhshrink())

            
        def forward(self, ref_batch, query_batch):
            ref_batch = self.ref_encoder(ref_batch)
            query_batch = self.query_encoder(query_batch)
            combined_batch = torch.cat((ref_batch, query_batch), 1)
            combined_batch = self.combined_encoder(combined_batch)
            ref_batch = self.ref_decoder(combined_batch)
            query_batch = self.query_decoder(combined_batch)
            return ref_batch, query_batch
        

    def train_test(self, r_hd, q_hd, r_bd, q_bd, c_hd, c_bd, batch_size, n_epochs, lr):
        
        ref_len = len(np.ravel(self.ref_data_t[0]))
        query_len = len(np.ravel(self.query_data_t[0]))
        ref_batches, query_batches = batchify_autoencoder(self.ref_data_t, self.query_data_t, batch_size)
        neural_network = RNA_seq_ae.Autoencoder(ref_len, query_len, r_hd, q_hd, r_bd, q_bd, c_hd, c_bd)
        optimizer = optim.Adagrad(neural_network.parameters(), lr=lr)
        loss_function = nn.MSELoss()
        neural_network.train()
        
        for i in range(n_epochs):
            error = 0
            for ii in range(len(ref_batches)):
                optimizer.zero_grad()
                ref_batch = ref_batches[ii]
                query_batch = query_batches[ii]
                predictions_ref, predictions_query = neural_network(torch.tensor(np.asarray(ref_batch).astype(np.float32)), torch.tensor(np.asarray(query_batch).astype(np.float32)))
                loss_ref = loss_function(predictions_ref, torch.tensor(np.asarray(ref_batch).astype(np.float32)))
                loss_query = loss_function(predictions_query, torch.tensor(np.asarray(query_batch).astype(np.float32)))
                loss = loss_ref + loss_query                   
                loss.backward()      
                optimizer.step()
                error += loss.data

            if i%1000 == 0:
                print("Epoch", i, ": Loss =", error) 
            
        return neural_network

## Helper Functions

Run the chunk below to create helper functions that will assist with data importation, batch creation, embedding generation, and the visualization of results

In [193]:
def data_import_sc(ref_sc, query_sc):

        ref_data = ref_sc
        query_data = query_sc

        sc.pp.filter_cells(ref_data, min_genes = 200)
        sc.pp.filter_genes(ref_data, min_cells = 3)

        sc.pp.filter_cells(query_data, min_genes = 200)
        sc.pp.filter_genes(query_data, min_cells = 3)

        ref_len = len(ref_data.obs)
        query_len = len(query_data.obs)

        var_names = ref_data.var_names.intersection(query_data.var_names)
        ref_data = ref_data[:, var_names]
        query_data = query_data[:, var_names]

        return ref_data, query_data
    
    
def batchify_autoencoder(ref, query, batch_size=16):
    
    query_batches = []
    ref_batches = []
    for n in range(0, len(query),batch_size):
        if n+batch_size < len(query):
            query_batch = query[n:n+batch_size]
            ref_batch = ref[n:n+batch_size]
            query_batches.append(query_batch)
            ref_batches.append(ref_batch)
            
    if len(query)%batch_size > 0:
        query_batch = query[len(query)-(len(query)%batch_size):len(query)]
        ref_batch = ref[len(ref)-(len(ref)%batch_size):len(ref)]
        query_batches.append(query_batch)
        ref_batches.append(ref_batch)
  
    return ref_batches, query_batches


def create_embed_df(matrix):
    df = pd.DataFrame()
    for col in range(matrix.shape[1]):
        df["em_" + str(col+1)] = matrix[:,col].detach().numpy()

    return df


def create_embeddings(model, ref_ann, query_ann, n_pca):
    ref = torch.from_numpy(ref_ann.X)
    query = torch.from_numpy(query_ann.X)
    ref_t = np.transpose(torch.from_numpy(ref_ann.X))
    query_t = np.transpose(torch.from_numpy(query_ann.X))
    ref_em = model.ref_encoder(ref_t)
    query_em = model.query_encoder(query_t)
    com = torch.cat((ref_em, query_em), 1)
    com = model.combined_encoder(com)
    pca_model = PCA(n_components=n_pca)
    ref_pca = pca_model.fit_transform(ref)
    query_pca = pca_model.fit_transform(query)
    com_df = create_embed_df(com)

    query_embeds = pd.DataFrame(np.dot(query_pca, np.transpose(com_df)))
    ref_embeds = pd.DataFrame(np.dot(ref_pca, np.transpose(com_df)))
    return ref_embeds, query_embeds

## Model Initialization

Run the chunk below to initialize the model with the specified data sets, and to return preprocessed data that will be used later

In [316]:
adata_ref = sc.datasets.pbmc3k_processed()
adata = sc.datasets.pbmc68k_reduced()
sc_map_test = RNA_seq_ae(adata_ref, adata)
ref_ann, query_ann = sc_map_test.processed_data()

In [264]:
models = []
r_hd = [1000, 1000, 1000, 1000]
q_hd = [600, 600, 600, 600]
r_bd = [200, 200, 200, 200]
q_bd = [100, 100, 100, 100]
c_hd = [100, 100, 100, 100]
c_bd = [3, 5, 10, 20]
for p1, p2, p3, p4, p5, p6 in zip(r_hd, q_hd, r_bd, q_bd, c_hd, c_bd):
    m = sc_map_test.train_test(r_hd=p1, q_hd=p2, r_bd=p3, q_bd=p4, c_hd=p5, c_bd=p6, batch_size = 16, n_epochs = 3000, lr=.01)
    models.append(m)
    print("Finished model")

Epoch 0 : Loss = tensor(38.4837)
Epoch 1000 : Loss = tensor(8.1753)
Epoch 2000 : Loss = tensor(5.6159)
Finished model
Epoch 0 : Loss = tensor(45.4452)
Epoch 1000 : Loss = tensor(4.9985)
Epoch 2000 : Loss = tensor(2.7435)
Finished model
Epoch 0 : Loss = tensor(45.4615)
Epoch 1000 : Loss = tensor(3.4567)
Epoch 2000 : Loss = tensor(1.9989)
Finished model
Epoch 0 : Loss = tensor(32.3752)
Epoch 1000 : Loss = tensor(2.4176)
Epoch 2000 : Loss = tensor(1.3412)
Finished model


## Results

Run the chunk below to produce the results of nearest neighbors clustering on the corrected cell count matrix, and the one below that for the results on the original cell count matrix. While multiple models were trained, only one's results are demonstrated below. 

In [325]:
adata_ref = sc.datasets.pbmc3k_processed()
adata = sc.datasets.pbmc68k_reduced()
sc_map_test = RNA_seq_ae(adata_ref, adata)
ref_ann, query_ann = sc_map_test.processed_data()

# create ground truth labels for query cells, and merge similar cell types
ref_y = list(ref_ann.obs['louvain'])
for i in range(0, len(ref_y)):
    if ref_y[i] in ['CD14+ Monocytes', 'FCGR3A+ Monocytes']:
        ref_y[i] = 'Monocytes'

ref_embeds, query_embeds = create_embeddings(models[1], ref_ann, query_ann, 5)
ref_ann.X = ref_embeds
ref_ann.obs['louvain'] = ref_y
query_ann.X = query_embeds
sc.pp.pca(ref_ann)
sc.pp.neighbors(ref_ann)
sc.tl.umap(ref_ann)
sc.tl.ingest(query_ann, ref_ann, obs='louvain')

query_y = list(query_ann.obs['bulk_labels'])
for i in range(0, len(query_y)):
    if query_y[i] in ['CD4+/CD25 T Reg', 'CD4+/CD45RA+/CD25- Naive T', 'CD4+/CD45RO+ Memory']:
        query_y[i] = 'CD4 T cells'
    elif query_y[i] in ['CD8+ Cytotoxic T', 'CD8+/CD45RA+ Naive Cytotoxic']:
        query_y[i] = 'CD8 T cells'
    elif query_y[i] in ['CD14+ Monocytes']:
        query_y[i] = 'Monocytes'
    elif query_y[i] in ['CD19+ B']:
        query_y[i] = 'B cells'
    elif query_y[i] == 'Dendritic':
        query_y[i] = 'Dendritic cells'
    elif query_y[i] in ['CD56+ NK']:
        query_y[i] = "NK cells"

query_ann.obs['bulk_labels'] = query_y
accuracy_score(list(query_ann.obs['bulk_labels']), list(query_ann.obs['louvain']))

Trying to set attribute `.obs` of view, copying.


0.25675675675675674

In [326]:
adata_ref = sc.datasets.pbmc3k_processed()
adata = sc.datasets.pbmc68k_reduced()
sc_map_test = RNA_seq_ae(adata_ref, adata)
ref_ann, query_ann = sc_map_test.processed_data()

ref_y = list(ref_ann.obs['louvain'])
for i in range(0, len(ref_y)):
    if ref_y[i] in ['CD14+ Monocytes', 'FCGR3A+ Monocytes']:
        ref_y[i] = 'Monocytes'

ref_ann.obs['louvain'] = ref_y
sc.pp.pca(ref_ann)
sc.pp.neighbors(ref_ann)
sc.tl.umap(ref_ann)
sc.tl.ingest(query_ann, ref_ann, obs='louvain')

query_y = list(query_ann.obs['bulk_labels'])
for i in range(0, len(query_y)):
    if query_y[i] in ['CD4+/CD25 T Reg', 'CD4+/CD45RA+/CD25- Naive T', 'CD4+/CD45RO+ Memory']:
        query_y[i] = 'CD4 T cells'
    elif query_y[i] in ['CD8+ Cytotoxic T', 'CD8+/CD45RA+ Naive Cytotoxic']:
        query_y[i] = 'CD8 T cells'
    elif query_y[i] in ['CD14+ Monocytes']:
        query_y[i] = 'Monocytes'
    elif query_y[i] in ['CD19+ B']:
        query_y[i] = 'B cells'
    elif query_y[i] == 'Dendritic':
        query_y[i] = 'Dendritic cells'
    elif query_y[i] in ['CD56+ NK']:
        query_y[i] = "NK cells"

query_ann.obs['bulk_labels'] = query_y
accuracy_score(list(query_ann.obs['bulk_labels']), list(query_ann.obs['louvain']))

Trying to set attribute `.obs` of view, copying.


0.22635135135135134